# ANN_Hyperparameter_Auto_Selection

Written by: Ketong Shao and Angelo D. Bonzanini

Date: October 2021

* Preprocessing the data
* Integrate Bayesian optimization and manulally coded 5-fold cross-validation 
* Automate the hyperparameter selection process of DNN

## Imports

In [13]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops

seed=0
np.random.seed(seed) # fix random seed
tf.random.set_seed(seed)

from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.layers import Dense, Dropout, Flatten

from sklearn.preprocessing import MinMaxScaler

In [3]:
# Load the experimental data
df = pd.read_csv('pintopindata.csv')

In [4]:
# print the data table to show how it looks likt
df

,mA,O2frac,mm,slm,Vdis,diluted,NO,NO2,HONO,number,R2
0,20.0,0.019608,2.0,0.5,0.552,0.0,208.807956,8.901428,1.248881e-01,5.00,0.740538
1,20.0,0.019608,2.0,1.0,0.547,0.0,84.002807,3.968063,2.241469e+00,5.00,0.881645
2,20.0,0.019608,2.0,3.0,0.554,0.0,54.087828,3.645636,3.710060e+00,5.00,0.678390
3,20.0,0.019608,5.0,0.5,0.850,0.0,651.411878,30.844485,9.414372e+00,5.00,0.661888
4,20.0,0.019608,5.0,1.0,0.880,0.0,322.280856,6.829662,1.370544e+00,4.50,0.769834
...,...,...,...,...,...,...,...,...,...,...,...
434,70.0,0.209500,15.0,7.0,1.330,0.0,1546.825335,89.786600,2.322819e-10,5.00,0.958551
435,70.0,0.500000,8.0,0.5,0.830,0.0,1605.305917,2620.939302,1.047542e+03,5.00,0.987230
436,70.0,0.500000,8.0,1.0,0.850,0.0,1780.145258,1483.692780,3.141424e+02,5.00,0.994900
437,70.0,0.500000,8.0,3.0,0.888,0.0,1579.088727,466.315225,3.947207e+00,5.00,0.995298


## Data Preprocessing

In [5]:
# calculate the discharge power
outpower = (np.array(df['Vdis'])*np.array(df['mA'])).reshape(-1,1)

# calculate the NOx production rate
outproduction = np.sum(np.array(df[['NO','NO2','HONO']]),axis=1).reshape(-1,1)*((np.array(df['slm'])+np.array(df['diluted'])).reshape(-1,1))

In [6]:
# build the outputs data
outputs = np.concatenate((outproduction,outpower),axis=1)
# extract the operating experimental parameters as inputs
inputs = np.array(df[['mA','O2frac','mm','slm']])

In [7]:
# shuffle the data, which is necessary for machine learning model training
np.random.seed(42)
index = list(range(len(inputs)))
np.random.shuffle(index)

outputs = outputs[index,:]
inputs = inputs[index,:]

In [8]:
# Definition of custom activation function 'swish'

from tensorflow.keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.layers import Activation

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

get_custom_objects().update({'swish': Activation(swish)})

## Bayesian optimization for DNN hyperparameter selection

In [11]:
# Necessary packages installation and importing
! pip install scikit-optimize
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer 

In [9]:
# Define DNN model structure
# Here, the nodes of hidden layers are all the same in order to reduce the number of hyperparameters to tune. 
# The DNN structrue can be modified and more complex.

from keras.layers import advanced_activations
def get_model_layers(nodes,layers,activation,learning_rate):
    seed = 42
    np.random.seed(seed)
    tf.random.set_seed(seed)
    model = Sequential()
    input_dims = 4 #this must match the number of input variables
    output_dim = 2
    if activation=='swish':
        activation = swish
    else:
        activation = activation
        
    inputs = tf.keras.layers.Input(shape=(input_dims,))
    x = tf.keras.layers.Dense(units=nodes, activation= activation)(inputs)
    for i in range(layers-1): #at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation= activation)(x)
    
    outputs = tf.keras.layers.Dense(output_dim, activation="linear")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adam(learning_rate=learning_rate,amsgrad=False)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error')
    return model

In [12]:
# Therefore, the hyperparameters are, the number of nodes of hidden layers, the number of hidden layers
# the activation function, the learning rate, the batch size and the epoch number

dim_learning_rate = Real(low=1e-6, high=0.1, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=20, name='num_layers') #reduced from 20 to 16...would like to reduce more
dim_num_dense_nodes = Integer(low=2, high=20, name='num_nodes')  #reduced from 20 to 16 basis my grid search
dim_activation = Categorical(categories=['relu','swish','tanh','sigmoid','selu','softplus','softsign','elu'],
                             name='activation')
dim_batch_size = Integer(low=4, high=300, name='batch_size')  #from 2 to 20 to 4 to 10
dim_epochs = Integer(low=100, high=8000, prior='log-uniform',name='epochs') #dropped from 7000 to 5000

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_dense_nodes,
              dim_activation,
              dim_batch_size,
              dim_epochs
             ]
# initial value, from which the Bayesian optimization starts
default_parameters = [0.007787502691691069, 7, 14,'relu', 200, 4473]

In [14]:
# Define the objective to be minimized by Bayesian Optimization.
# Here, the objective is the mean squared error by true value and prediction. 
@use_named_args(dimensions=dimensions) #method called decorator
def loss_obj(learning_rate,num_layers,num_nodes,activation,batch_size,epochs):
    
    epochs = epochs
    batch_size = batch_size
    
    NNmodel = get_model_layers(nodes=num_nodes, layers=num_layers,
                               activation=activation, learning_rate=learning_rate)   
    
    
    X = inputs
    Y = outputs

    nn_test_mse = np.zeros((len(Y),1))


    length = X.shape[0]
    index= int(np.ceil(length/5))
    
    # manually coded 5-fold cross-validation
    
    def createList(r1, r2): 
        return [item for item in range(r1, r2)] 
    index0 = createList(0,index)
    index1 = createList(index,2*index)
    index2 = createList(2*index,3*index)
    index3 = createList(3*index,4*index)
    index4 = createList(4*index,length)

    index_set = [index0, index1, index2, index3, index4]    
    
    for cv in range(5):

        training_label_1 = Y[index_set[cv-1],:]
        training_label_2 = Y[index_set[cv-2],:]
        training_label_3 = Y[index_set[cv-3],:]
        training_label_4 = Y[index_set[cv-4],:]
        training_label = np.concatenate((training_label_1,training_label_2,training_label_3,training_label_4),0)
        test_label = Y[index_set[cv],:]
        
        scaler_label = MinMaxScaler()
        scaler_label.fit(training_label)
        training_label = scaler_label.transform(training_label)
        test_label = scaler_label.transform(test_label)

        training_set_1 = X[index_set[cv-1],:]
        training_set_2 = X[index_set[cv-2],:]
        training_set_3 = X[index_set[cv-3],:]
        training_set_4 = X[index_set[cv-4],:]
        training_set = np.concatenate((training_set_1,training_set_2,training_set_3,training_set_4),0)
        test_set = X[index_set[cv],:]

        scaler_data = MinMaxScaler()
        scaler_data.fit(training_set)
        training_set = scaler_data.transform(training_set)
        test_set = scaler_data.transform(test_set)

        
        print(training_set.shape)
        

        NNmodel.fit(training_set,training_label,
                    batch_size = batch_size,
                    epochs= epochs,
                    verbose = 2)

        val = NNmodel.predict(test_set)
        
        val = scaler_label.inverse_transform(val)
        truth = scaler_label.inverse_transform(test_label)
        nn_test_mse[index_set[cv],0] = np.mean((val-truth)*(val-truth),axis=1)
    
    
    final_mse = np.mean(nn_test_mse)
    
    print('New GP iteration with MSE : ', final_mse)

    # Print the MSE.
    print()
    print("MSE: {0:.2%}".format(final_mse))
    print()

    # Delete the Keras model with these hyper-parameters from memory.
    del NNmodel
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    ops.reset_default_graph()

    return score

In [ ]:
#dimensions = [dim_learning_rate,
#              dim_num_dense_layers,
#              dim_num_dense_nodes,
#              dim_activation,
#              dim_batch_size,
#              dim_epochs
#             ]

# use CheckpointSaver to save the optimization process in case of any interruption 
# This Bayesian optimization might take some time.
from skopt.callbacks import CheckpointSaver

checkpoint_saver = CheckpointSaver("./checkpoint.pkl", compress=9) # keyword arguments will be passed to `skopt.dump`

# last_res = skopt.load('checkpoint.pkl')
gp_result = gp_minimize(func=loss_obj,
                            dimensions=dimensions,
                            n_calls=100, 
                            noise= 1e-8,
                            n_jobs=-1,
                            acq_func="EI",
                            x0=default_parameters,
                            random_state=42,
                            #if starting from last checkpoint, comment out x0 = default_parameters,
                            #add x = last_res.x_iters, y = last_res.func_vals
                            callback=[checkpoint_saver],)
opt_h = gp_result.x[0],gp_result.x[1],gp_result.x[2],gp_result.x[3],gp_result.x[4],gp_result.x[5]
print(opt_h)
print(gp_result.fun)

In [ ]:
# save the finished Bayesian optimization
# skopt.dump(gp_result,'BO_result')

In [21]:
# Here we directly load the completed Bayesian Optimization, with 674 evaluations.
# Note, you may obtain different hyperparameter values, but the score should be close.
BO_result = skopt.load('BO_for_DNN_res')

# Print the found best MSE
print('Bestf found MSE:', BO_result.fun)
print('-----------------------------------------------')
# Print the corresponding hyperparameter values
print('Hyperparameter values:', BO_result.x)
print('-----------------------------------------------')
print('Corresponding to:','\n',
      'Learning rate:', BO_result.x[0],'\n',
      'Number of hidden layers:', BO_result.x[1],'\n',
      'Number of nodes for each hidden layer:', BO_result.x[2],'\n',
      'Activation function:', BO_result.x[3],'\n',
      'Batch size:', BO_result.x[4],'\n',
      'Epoch number:', BO_result.x[5],'\n',)

Bestf found MSE: 6531.132583325251
-----------------------------------------------
Hyperparameter values: [0.004764094298461491, 5, 11, 'swish', 191, 5684]
-----------------------------------------------
Corresponding to: 
 Learning rate: 0.004764094298461491 
 Number of hidden layers: 5 
 Number of nodes for each hidden layer: 11 
 Activation function: swish 
 Batch size: 191 
 Epoch number: 5684 



## DNN training using the found hyperparameters

In [22]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_data = scaler_x.fit_transform(inputs)
Y_data = scaler_y.fit_transform(outputs)

final_model = get_model_layers(nodes=11,layers=5,activation = 'swish',
                                          learning_rate = 0.004764094298461491)
final_model.fit(X_data,Y_data,
                batch_size = 191,
                epochs = 5684,
                verbose = 2)

Epoch 1/5684
3/3 - 0s - loss: 0.1402
Epoch 2/5684
3/3 - 0s - loss: 0.1120
Epoch 3/5684
3/3 - 0s - loss: 0.0820
Epoch 4/5684
3/3 - 0s - loss: 0.0542
Epoch 5/5684
3/3 - 0s - loss: 0.0368
Epoch 6/5684
3/3 - 0s - loss: 0.0394
Epoch 7/5684
3/3 - 0s - loss: 0.0402
Epoch 8/5684
3/3 - 0s - loss: 0.0335
Epoch 9/5684
3/3 - 0s - loss: 0.0314
Epoch 10/5684
3/3 - 0s - loss: 0.0304
Epoch 11/5684
3/3 - 0s - loss: 0.0281
Epoch 12/5684
3/3 - 0s - loss: 0.0251
Epoch 13/5684
3/3 - 0s - loss: 0.0227
Epoch 14/5684
3/3 - 0s - loss: 0.0202
Epoch 15/5684
3/3 - 0s - loss: 0.0175
Epoch 16/5684
3/3 - 0s - loss: 0.0156
Epoch 17/5684
3/3 - 0s - loss: 0.0131
Epoch 18/5684
3/3 - 0s - loss: 0.0115
Epoch 19/5684
3/3 - 0s - loss: 0.0103
Epoch 20/5684
3/3 - 0s - loss: 0.0091
Epoch 21/5684
3/3 - 0s - loss: 0.0082
Epoch 22/5684
3/3 - 0s - loss: 0.0077
Epoch 23/5684
3/3 - 0s - loss: 0.0071
Epoch 24/5684
3/3 - 0s - loss: 0.0067
Epoch 25/5684
3/3 - 0s - loss: 0.0065
Epoch 26/5684
3/3 - 0s - loss: 0.0064
Epoch 27/5684
3/3 - 0

Epoch 207/5684
3/3 - 0s - loss: 6.8950e-04
Epoch 208/5684
3/3 - 0s - loss: 7.5421e-04
Epoch 209/5684
3/3 - 0s - loss: 8.0705e-04
Epoch 210/5684
3/3 - 0s - loss: 8.0572e-04
Epoch 211/5684
3/3 - 0s - loss: 7.2022e-04
Epoch 212/5684
3/3 - 0s - loss: 7.1615e-04
Epoch 213/5684
3/3 - 0s - loss: 7.0440e-04
Epoch 214/5684
3/3 - 0s - loss: 6.7627e-04
Epoch 215/5684
3/3 - 0s - loss: 6.6752e-04
Epoch 216/5684
3/3 - 0s - loss: 6.9850e-04
Epoch 217/5684
3/3 - 0s - loss: 6.9646e-04
Epoch 218/5684
3/3 - 0s - loss: 6.7782e-04
Epoch 219/5684
3/3 - 0s - loss: 7.0709e-04
Epoch 220/5684
3/3 - 0s - loss: 6.9635e-04
Epoch 221/5684
3/3 - 0s - loss: 6.6913e-04
Epoch 222/5684
3/3 - 0s - loss: 6.6319e-04
Epoch 223/5684
3/3 - 0s - loss: 6.7013e-04
Epoch 224/5684
3/3 - 0s - loss: 6.7056e-04
Epoch 225/5684
3/3 - 0s - loss: 6.7014e-04
Epoch 226/5684
3/3 - 0s - loss: 7.3611e-04
Epoch 227/5684
3/3 - 0s - loss: 6.9574e-04
Epoch 228/5684
3/3 - 0s - loss: 6.4762e-04
Epoch 229/5684
3/3 - 0s - loss: 6.5067e-04
Epoch 230/5

Epoch 398/5684
3/3 - 0s - loss: 4.5788e-04
Epoch 399/5684
3/3 - 0s - loss: 4.5164e-04
Epoch 400/5684
3/3 - 0s - loss: 4.7262e-04
Epoch 401/5684
3/3 - 0s - loss: 4.6210e-04
Epoch 402/5684
3/3 - 0s - loss: 4.7212e-04
Epoch 403/5684
3/3 - 0s - loss: 4.5808e-04
Epoch 404/5684
3/3 - 0s - loss: 4.4465e-04
Epoch 405/5684
3/3 - 0s - loss: 4.4598e-04
Epoch 406/5684
3/3 - 0s - loss: 4.4461e-04
Epoch 407/5684
3/3 - 0s - loss: 4.3950e-04
Epoch 408/5684
3/3 - 0s - loss: 4.4041e-04
Epoch 409/5684
3/3 - 0s - loss: 4.3581e-04
Epoch 410/5684
3/3 - 0s - loss: 4.4598e-04
Epoch 411/5684
3/3 - 0s - loss: 4.3984e-04
Epoch 412/5684
3/3 - 0s - loss: 4.4202e-04
Epoch 413/5684
3/3 - 0s - loss: 4.3456e-04
Epoch 414/5684
3/3 - 0s - loss: 4.5377e-04
Epoch 415/5684
3/3 - 0s - loss: 4.4400e-04
Epoch 416/5684
3/3 - 0s - loss: 4.5450e-04
Epoch 417/5684
3/3 - 0s - loss: 4.4399e-04
Epoch 418/5684
3/3 - 0s - loss: 4.7196e-04
Epoch 419/5684
3/3 - 0s - loss: 4.5893e-04
Epoch 420/5684
3/3 - 0s - loss: 4.9730e-04
Epoch 421/5

Epoch 589/5684
3/3 - 0s - loss: 4.0084e-04
Epoch 590/5684
3/3 - 0s - loss: 3.5119e-04
Epoch 591/5684
3/3 - 0s - loss: 3.7507e-04
Epoch 592/5684
3/3 - 0s - loss: 3.7091e-04
Epoch 593/5684
3/3 - 0s - loss: 3.4207e-04
Epoch 594/5684
3/3 - 0s - loss: 3.9616e-04
Epoch 595/5684
3/3 - 0s - loss: 3.6838e-04
Epoch 596/5684
3/3 - 0s - loss: 3.9645e-04
Epoch 597/5684
3/3 - 0s - loss: 3.7403e-04
Epoch 598/5684
3/3 - 0s - loss: 3.6228e-04
Epoch 599/5684
3/3 - 0s - loss: 3.7706e-04
Epoch 600/5684
3/3 - 0s - loss: 3.7368e-04
Epoch 601/5684
3/3 - 0s - loss: 4.4337e-04
Epoch 602/5684
3/3 - 0s - loss: 4.1383e-04
Epoch 603/5684
3/3 - 0s - loss: 3.9195e-04
Epoch 604/5684
3/3 - 0s - loss: 3.9449e-04
Epoch 605/5684
3/3 - 0s - loss: 3.7855e-04
Epoch 606/5684
3/3 - 0s - loss: 3.5183e-04
Epoch 607/5684
3/3 - 0s - loss: 3.6138e-04
Epoch 608/5684
3/3 - 0s - loss: 3.7317e-04
Epoch 609/5684
3/3 - 0s - loss: 3.4146e-04
Epoch 610/5684
3/3 - 0s - loss: 3.5050e-04
Epoch 611/5684
3/3 - 0s - loss: 3.5311e-04
Epoch 612/5

Epoch 780/5684
3/3 - 0s - loss: 2.6478e-04
Epoch 781/5684
3/3 - 0s - loss: 2.8438e-04
Epoch 782/5684
3/3 - 0s - loss: 2.8694e-04
Epoch 783/5684
3/3 - 0s - loss: 2.6735e-04
Epoch 784/5684
3/3 - 0s - loss: 2.8210e-04
Epoch 785/5684
3/3 - 0s - loss: 2.7242e-04
Epoch 786/5684
3/3 - 0s - loss: 2.7555e-04
Epoch 787/5684
3/3 - 0s - loss: 2.8308e-04
Epoch 788/5684
3/3 - 0s - loss: 2.7930e-04
Epoch 789/5684
3/3 - 0s - loss: 3.2055e-04
Epoch 790/5684
3/3 - 0s - loss: 3.1876e-04
Epoch 791/5684
3/3 - 0s - loss: 2.8418e-04
Epoch 792/5684
3/3 - 0s - loss: 2.9160e-04
Epoch 793/5684
3/3 - 0s - loss: 2.7255e-04
Epoch 794/5684
3/3 - 0s - loss: 2.7929e-04
Epoch 795/5684
3/3 - 0s - loss: 2.8519e-04
Epoch 796/5684
3/3 - 0s - loss: 2.7233e-04
Epoch 797/5684
3/3 - 0s - loss: 2.5718e-04
Epoch 798/5684
3/3 - 0s - loss: 2.5585e-04
Epoch 799/5684
3/3 - 0s - loss: 2.5230e-04
Epoch 800/5684
3/3 - 0s - loss: 2.5541e-04
Epoch 801/5684
3/3 - 0s - loss: 2.5528e-04
Epoch 802/5684
3/3 - 0s - loss: 2.6359e-04
Epoch 803/5

...

Epoch 4888/5684
3/3 - 0s - loss: 6.0552e-05
Epoch 4889/5684
3/3 - 0s - loss: 5.1607e-05
Epoch 4890/5684
3/3 - 0s - loss: 4.6862e-05
Epoch 4891/5684
3/3 - 0s - loss: 4.3394e-05
Epoch 4892/5684
3/3 - 0s - loss: 4.6596e-05
Epoch 4893/5684
3/3 - 0s - loss: 4.2263e-05
Epoch 4894/5684
3/3 - 0s - loss: 4.5498e-05
Epoch 4895/5684
3/3 - 0s - loss: 4.4925e-05
Epoch 4896/5684
3/3 - 0s - loss: 4.6183e-05
Epoch 4897/5684
3/3 - 0s - loss: 5.7457e-05
Epoch 4898/5684
3/3 - 0s - loss: 4.6889e-05
Epoch 4899/5684
3/3 - 0s - loss: 5.2710e-05
Epoch 4900/5684
3/3 - 0s - loss: 6.4139e-05
Epoch 4901/5684
3/3 - 0s - loss: 5.2184e-05
Epoch 4902/5684
3/3 - 0s - loss: 4.6534e-05
Epoch 4903/5684
3/3 - 0s - loss: 4.6863e-05
Epoch 4904/5684
3/3 - 0s - loss: 4.8510e-05
Epoch 4905/5684
3/3 - 0s - loss: 5.0502e-05
Epoch 4906/5684
3/3 - 0s - loss: 4.5101e-05
Epoch 4907/5684
3/3 - 0s - loss: 4.7944e-05
Epoch 4908/5684
3/3 - 0s - loss: 4.8330e-05
Epoch 4909/5684
3/3 - 0s - loss: 4.6820e-05
Epoch 4910/5684
3/3 - 0s - loss:

3/3 - 0s - loss: 4.5349e-05
Epoch 5075/5684
3/3 - 0s - loss: 4.2856e-05
Epoch 5076/5684
3/3 - 0s - loss: 4.2753e-05
Epoch 5077/5684
3/3 - 0s - loss: 4.7196e-05
Epoch 5078/5684
3/3 - 0s - loss: 4.1546e-05
Epoch 5079/5684
3/3 - 0s - loss: 4.6310e-05
Epoch 5080/5684
3/3 - 0s - loss: 4.5168e-05
Epoch 5081/5684
3/3 - 0s - loss: 4.5773e-05
Epoch 5082/5684
3/3 - 0s - loss: 4.7850e-05
Epoch 5083/5684
3/3 - 0s - loss: 4.5846e-05
Epoch 5084/5684
3/3 - 0s - loss: 4.9834e-05
Epoch 5085/5684
3/3 - 0s - loss: 4.7496e-05
Epoch 5086/5684
3/3 - 0s - loss: 5.0473e-05
Epoch 5087/5684
3/3 - 0s - loss: 4.5106e-05
Epoch 5088/5684
3/3 - 0s - loss: 4.0453e-05
Epoch 5089/5684
3/3 - 0s - loss: 4.1508e-05
Epoch 5090/5684
3/3 - 0s - loss: 4.4473e-05
Epoch 5091/5684
3/3 - 0s - loss: 4.2996e-05
Epoch 5092/5684
3/3 - 0s - loss: 4.9482e-05
Epoch 5093/5684
3/3 - 0s - loss: 4.6983e-05
Epoch 5094/5684
3/3 - 0s - loss: 4.5402e-05
Epoch 5095/5684
3/3 - 0s - loss: 4.8683e-05
Epoch 5096/5684
3/3 - 0s - loss: 4.0164e-05
Epoc

Epoch 5261/5684
3/3 - 0s - loss: 3.7967e-05
Epoch 5262/5684
3/3 - 0s - loss: 3.9005e-05
Epoch 5263/5684
3/3 - 0s - loss: 3.5049e-05
Epoch 5264/5684
3/3 - 0s - loss: 3.7268e-05
Epoch 5265/5684
3/3 - 0s - loss: 3.7107e-05
Epoch 5266/5684
3/3 - 0s - loss: 3.9164e-05
Epoch 5267/5684
3/3 - 0s - loss: 3.9466e-05
Epoch 5268/5684
3/3 - 0s - loss: 6.1545e-05
Epoch 5269/5684
3/3 - 0s - loss: 4.7487e-05
Epoch 5270/5684
3/3 - 0s - loss: 4.5366e-05
Epoch 5271/5684
3/3 - 0s - loss: 5.0707e-05
Epoch 5272/5684
3/3 - 0s - loss: 4.3838e-05
Epoch 5273/5684
3/3 - 0s - loss: 4.1929e-05
Epoch 5274/5684
3/3 - 0s - loss: 3.5643e-05
Epoch 5275/5684
3/3 - 0s - loss: 3.5787e-05
Epoch 5276/5684
3/3 - 0s - loss: 3.8601e-05
Epoch 5277/5684
3/3 - 0s - loss: 4.0617e-05
Epoch 5278/5684
3/3 - 0s - loss: 4.0386e-05
Epoch 5279/5684
3/3 - 0s - loss: 3.7545e-05
Epoch 5280/5684
3/3 - 0s - loss: 3.8643e-05
Epoch 5281/5684
3/3 - 0s - loss: 4.8286e-05
Epoch 5282/5684
3/3 - 0s - loss: 5.0018e-05
Epoch 5283/5684
3/3 - 0s - loss:

3/3 - 0s - loss: 5.7901e-05
Epoch 5448/5684
3/3 - 0s - loss: 5.4561e-05
Epoch 5449/5684
3/3 - 0s - loss: 5.7137e-05
Epoch 5450/5684
3/3 - 0s - loss: 4.8785e-05
Epoch 5451/5684
3/3 - 0s - loss: 5.4462e-05
Epoch 5452/5684
3/3 - 0s - loss: 4.7847e-05
Epoch 5453/5684
3/3 - 0s - loss: 4.5144e-05
Epoch 5454/5684
3/3 - 0s - loss: 4.8224e-05
Epoch 5455/5684
3/3 - 0s - loss: 5.7354e-05
Epoch 5456/5684
3/3 - 0s - loss: 4.5497e-05
Epoch 5457/5684
3/3 - 0s - loss: 5.2427e-05
Epoch 5458/5684
3/3 - 0s - loss: 5.5329e-05
Epoch 5459/5684
3/3 - 0s - loss: 5.3997e-05
Epoch 5460/5684
3/3 - 0s - loss: 5.5967e-05
Epoch 5461/5684
3/3 - 0s - loss: 4.4950e-05
Epoch 5462/5684
3/3 - 0s - loss: 4.3884e-05
Epoch 5463/5684
3/3 - 0s - loss: 4.3977e-05
Epoch 5464/5684
3/3 - 0s - loss: 4.1791e-05
Epoch 5465/5684
3/3 - 0s - loss: 4.8482e-05
Epoch 5466/5684
3/3 - 0s - loss: 4.5155e-05
Epoch 5467/5684
3/3 - 0s - loss: 4.4109e-05
Epoch 5468/5684
3/3 - 0s - loss: 5.0402e-05
Epoch 5469/5684
3/3 - 0s - loss: 4.8354e-05
Epoc

Epoch 5634/5684
3/3 - 0s - loss: 3.5780e-05
Epoch 5635/5684
3/3 - 0s - loss: 3.4462e-05
Epoch 5636/5684
3/3 - 0s - loss: 3.5976e-05
Epoch 5637/5684
3/3 - 0s - loss: 4.6585e-05
Epoch 5638/5684
3/3 - 0s - loss: 4.4085e-05
Epoch 5639/5684
3/3 - 0s - loss: 3.7404e-05
Epoch 5640/5684
3/3 - 0s - loss: 3.9276e-05
Epoch 5641/5684
3/3 - 0s - loss: 3.7993e-05
Epoch 5642/5684
3/3 - 0s - loss: 3.6246e-05
Epoch 5643/5684
3/3 - 0s - loss: 3.6973e-05
Epoch 5644/5684
3/3 - 0s - loss: 4.2003e-05
Epoch 5645/5684
3/3 - 0s - loss: 4.1916e-05
Epoch 5646/5684
3/3 - 0s - loss: 4.7817e-05
Epoch 5647/5684
3/3 - 0s - loss: 4.4573e-05
Epoch 5648/5684
3/3 - 0s - loss: 4.3420e-05
Epoch 5649/5684
3/3 - 0s - loss: 4.1595e-05
Epoch 5650/5684
3/3 - 0s - loss: 4.7408e-05
Epoch 5651/5684
3/3 - 0s - loss: 4.1362e-05
Epoch 5652/5684
3/3 - 0s - loss: 5.0343e-05
Epoch 5653/5684
3/3 - 0s - loss: 5.0248e-05
Epoch 5654/5684
3/3 - 0s - loss: 6.0559e-05
Epoch 5655/5684
3/3 - 0s - loss: 5.2394e-05
Epoch 5656/5684
3/3 - 0s - loss:

In [23]:
# Save the model after training.
final_model.save('Pintopin_DNN_model')

INFO:tensorflow:Assets written to: Pintopin_DNN_model/assets
